# Titanic - Feature Engineering!
In this notebook, we will begin engineering features appropriate to build our machine learning model 

In [2]:
# Importing the necessary Python libraries
import pandas as pd

In [3]:
# Loading in the training dataset
df_train = pd.read_csv('../data/train.csv')

## What We Did Last Time...

In [4]:
# Dropping unnecessary columns
df_train.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

In [5]:
# Changing Sex (gender) to numerical values
df_train['Sex'].mask(df_train['Sex'] == 'male', 0, inplace = True)
df_train['Sex'].mask(df_train['Sex'] == 'female', 1, inplace = True)
df_train['Sex'] = df_train['Sex'].astype(int)

In [6]:
# Separating the supporting features (X) from the predictor feature (y)
X = df_train.drop(columns = ['Survived'])
y = df_train[['Survived']]

## Starting our Feature Engineering!